In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/GitHub

/content/drive/MyDrive/GitHub


In [3]:
repository = "NAS_project"

In [4]:
%cd {repository}

/content/drive/MyDrive/GitHub/NAS_project


In [ ]:
pip install nats_bench

In [ ]:
pip install yacs

In [ ]:
pip install simplejson

In [ ]:
pip install xautodl

In [ ]:
pip install pytorchcv

In [10]:
%env TORCH_HOME=/content/drive/MyDrive/NAS-PULITO/.torch

env: TORCH_HOME=/content/drive/MyDrive/NAS-PULITO/.torch


In [11]:
# import librerie

# librerie già presenti nelle prime versioni
import torch
import argparse
import datasets 
import nasspace
import pandas as pd
import csv
import random
import numpy as np
from timeit import default_timer as timer
import os
from scipy import stats
from nats_bench import create
import xautodl  # import this lib -- "https://github.com/D-X-Y/AutoDL-Projects", you can use pip install xautodl
from xautodl.models import get_cell_based_tiny_net
from sklearn.preprocessing import StandardScaler
from numpy import inf

#librerie nuove
from pycls.models.nas.nas import Cell
import pandas as pd
from pytorchcv.model_provider import get_model as ptcv_get_model
from pytorchcv.model_provider import _models as ptcv_models
from pruners import *
from pruners import predictive
from tqdm import trange
from statistics import mean
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
from timeit import default_timer as timer

from GenNAS.builder_task import *
from GenNAS.builder_model import *
from GenNAS.builder_evaluator import *
from GenNAS.utils.config_generator import *


In [13]:
#SELECT DATASET
#Dataset_name = "Cifar-10"
#Dataset_name = "Cifar-100"
Dataset_name = "ImageNet"

In [14]:
GPU = '0'
seed = 1
batch_size = 128

#CIFAR10
if Dataset_name == "Cifar-10":
  dataset = 'cifar10'
  data_loc = '/content/drive/MyDrive/Github/NAS-PULITO/cifar-10-batches-py'


#CIFAR100
if Dataset_name == "Cifar-100":
  dataset = 'cifar100'
  data_loc = "/content/drive/MyDrive/NAS-PULITO/cifar-100-python"

#IMAGENET
if Dataset_name == "ImageNet":
  dataset = 'ImageNet16-120'
  data_loc = "/content/drive/MyDrive/NAS/.torch/ImageNet16"



In [15]:
os.environ['CUDA_VISIBLE_DEVICES'] = GPU

# Reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [16]:
from timeit import default_timer as timer
import torch


def get_naswot(network, x):
  network = network.to(device)

  def boolRelu_forward_hook(module, inp, out):
    if isinstance(out, tuple):
      out = out[0]
    out = out.view(out.size(0), -1)   
    x = (out > 0).float()  
    network.boolRelu = torch.cat((network.boolRelu, x), -1)
    
  network.boolRelu = torch.tensor([])
  network.boolRelu = network.boolRelu.to(device)

  for name, module in network.named_modules(): 
    if (isinstance(module, torch.nn.modules.activation.ReLU)):
      module.register_forward_hook(boolRelu_forward_hook)
              

  network(x)
  k = (network.boolRelu @ network.boolRelu.t()) + ((1. - network.boolRelu) @ (1. - network.boolRelu.t()))
  logdet = torch.linalg.slogdet(k)[1].cpu().detach()
  torch.cuda.empty_cache()
  return logdet
  #return (logdet, (end - start))

In [17]:
def NNDegree(searchspace, uid):

  strOp = searchspace.get_str(uid)
  listOp = strOp.split('|')
  id = [1,4,8]
  nConv = strOp.count('conv')
  skip1 = [3,7]
  skip2 = [6]
  nID = ''.join([i for j, i in enumerate(listOp) if j in id]).count('skip')
  nS1 = ''.join([i for j, i in enumerate(listOp) if j in skip1]).count('skip')
  nS2  = ''.join([i for j, i in enumerate(listOp) if j in skip2]).count('skip')
  nNone = strOp.count('none')
  #skip = searchspace.get_unique_str(uid).count('skip')
  Sc = (nS1+2*nS2)
  Wc = nConv
  return int((Wc + Sc))

In [18]:
from sklearn.preprocessing import StandardScaler
from numpy import inf

def get_uid_and_measure(uid, input, metric_name, population = None):
  scaler = StandardScaler()
  uid = int(uid)
  network = searchspace.get_network(uid)
  network.to(device)
  if metric_name == "Naswot":
    return uid, get_naswot(network, input).item()
  elif metric_name == "log_synflow":
    deg =  NNDegree(searchspace, uid)
    return uid, predictive.get_log_syn(network, input), deg
  elif metric_name == "normalize":
    naswot = get_naswot(network, train_loader).item()
    deg =  NNDegree(searchspace, uid)
    p = population[:,[0,2,3]]
    #print("p: ", p)
    #print("np.array([uid, degree, naswot]): ", np.array([uid, naswot, deg]))
    p = np.vstack((p, np.array([uid, naswot, deg])))
    #print(p)
    p[p == -inf] = 0
    NWnor = scaler.fit_transform(p[:,1].reshape(-1,1))
    degree = scaler.fit_transform(p[:,2].reshape(-1,1))
    return (uid , NWnor[-1].item()+degree[-1].item(),  naswot, deg)

  else:
    return 0

In [19]:
from sklearn.preprocessing import StandardScaler
from numpy import inf
 #uid, sum, naswot, deg, acc

def sum_syn_naswot_degree(list_measures, input, searchspace, task, model_builder, evaluator):
  nas = []
  slope = []
  scaler = StandardScaler()
  list_measures[list_measures == -inf] = 0
  syn = list_measures[:,1].reshape(-1,1)
  #deg = scaler.fit_transform(list_measures[:,2].reshape(-1,1))
  syn_nor = scaler.fit_transform(list_measures[:,1].reshape(-1,1))
  degree = list_measures[:,2].reshape(-1,1)
  deg = scaler.fit_transform(list_measures[:,2].reshape(-1,1))
  for uid in list_measures[:,0]:
      network = searchspace.get_network(int(uid))
      network.to(device)
      nas.append([ get_naswot(network, input).item()])
      #slope.append([get_regression_score(int(uid), task, model_builder,evaluator)])
  #return (list_measures[:,0], NWnor, degree, syn)
  naswot = np.array(nas).reshape(-1,1)
  WNor = scaler.fit_transform(np.array(nas).reshape(-1,1))
  #sl = scaler.fit_transform(np.array(slope).reshape(-1,1))
  measures = np.hstack((list_measures[:,0].reshape(-1,1), syn_nor+WNor+deg, syn, naswot, degree)) #NWnor+degree+syn+
  return measures

In [20]:
def get_uid_and_measures(uid_list, input, searchspace, metric_name):
  naswot = []
  log_Syn = []
  norm = []

  if metric_name == "Naswot":
    for uid in uid_list:
      network = searchspace.get_network(int(uid))
      network.to(device)
      naswot.append([uid, get_naswot(network, input).item()])
    return np.array(naswot)
  elif metric_name == "log_synflow":
    for uid in uid_list:
      deg =  NNDegree(searchspace, uid)
      network = searchspace.get_network(int(uid))
      log_Syn.append([uid, predictive.get_log_syn(network, input), deg])
    return np.array(log_Syn)
  elif metric_name == "normalize":
    for uid in uid_list:
      network = searchspace.get_network(int(uid))
      network.to(device)
      #net2 = searchspace.get_network(int(uid))
      norm.append([uid, get_naswot(network, train_loader).item(), NNDegree(searchspace, uid)])
    return sum_normalize(np.array(norm))
  else:
    return 0

In [21]:
def get_archs(searchspace,list_uid):
  list_arch = []
  for uid in list_uid:
    list_arch.append(searchspace.get_unique_str(uid))
  return list_arch

In [22]:
def lossSlope(losslist):
  trend = np.polyfit(range(len(losslist[1:10])),losslist[1:10],1)
  return trend[0]
  #trendpoly = np.poly1d(trend) 
  #return trendpoly(len(losslist)+10)

In [23]:
def LAS_score(losslist):
  weight = np.logspace(0, 1, len(losslist[1:]), endpoint=True)
  return np.log(np.dot(losslist[1:], weight))#/weight.sum()  #- np.sqrt(len(losslist))*abs(np.polyfit(range(len(losslist)),losslist,1)[0])

In [24]:
def LAS_score_trp(losslist):
  return np.trapz(losslist, dx= 0.1)

In [25]:
def tse_ema(losslist):
  mu = 0.70
  m = np.linspace(1, 0, num=len(losslist[1:]))
  for i, loss in enumerate(losslist[1:]):
    if i <= 0:
      ema = loss
    else:
      ema = ema * (1 - mu) + mu * loss 
      #ema = ema * (1 - m[i]) + m[i] * loss

  return ema

In [26]:
def LAS_score_g(losslist):
  return -np.sum((np.gradient(losslist[1:])))

In [27]:
def get_regression_score(uid, task, model_builder, evaluator):
  arch = searchspace.get_str(uid)
  losses = evaluator.evaluate(task, model_builder, arch)
  return -lossSlope(list(losses))
  #return -list(losses)[-1]
  #return (LAS_score_trp(list(losses)))
  #return (tse_ema(list(losses)))
  #return (-LAS_score_g(list(losses)))
  #return (-LAS_score(list(losses)))

In [28]:

from collections import defaultdict


def dominates(obj1, obj2, sign=[1, 1]):
    """Return true if each objective of *self* is not strictly worse than
            the corresponding objective of *other* and at least one objective is
            strictly better.
        **no need to care about the equal cases
        (Cuz equal cases mean they are non-dominators)
    :param obj1: a list of multiple objective values
    :type obj1: numpy.ndarray
    :param obj2: a list of multiple objective values
    :type obj2: numpy.ndarray
    :param sign: target types. positive means maximize and otherwise minimize.
    :type sign: list
    """
    indicator = False
    for a, b, sign in zip(obj1, obj2, sign):
        if a * sign > b * sign:
            indicator = True
        # if one of the objectives is dominated, then return False
        elif a * sign < b * sign:
            return False
    return indicator


def sortNondominated(fitness, k=None, first_front_only=False):
    """Sort the first *k* *individuals* into different nondomination levels
        using the "Fast Nondominated Sorting Approach" proposed by Deb et al.,
        see [Deb2002]_. This algorithm has a time complexity of :math:`O(MN^2)`,
        where :math:`M` is the number of objectives and :math:`N` the number of
        individuals.
        :param individuals: A list of individuals to select from.
        :param k: The number of individuals to select.
        :param first_front_only: If :obj:`True` sort only the first front and
                                    exit.
        :param sign: indicate the objectives are maximized or minimized
        :returns: A list of Pareto fronts (lists), the first list includes
                    nondominated individuals.
        .. [Deb2002] Deb, Pratab, Agarwal, and Meyarivan, "A fast elitist
            non-dominated sorting genetic algorithm for multi-objective
            optimization: NSGA-II", 2002.
    """
    fitness = np.ndarray.tolist(fitness)
    if k is None:
        k = len(fitness)

    # Use objectives as keys to make python dictionary
    map_fit_ind = defaultdict(list)
    for i, f_value in enumerate(fitness):  # fitness = [(1, 2), (2, 2), (3, 1), (1, 4), (1, 1)...]
        map_fit_ind[tuple(f_value)].append(i)
    fits = list(map_fit_ind.keys())  # fitness values

    current_front = []
    next_front = []
    dominating_fits = defaultdict(int)  # n (The number of people dominate you)
    dominated_fits = defaultdict(list)  # Sp (The people you dominate)

    # Rank first Pareto front
    # *fits* is a iterable list of chromosomes. Each has multiple objectives.
    for i, fit_i in enumerate(fits):
        for fit_j in fits[i + 1:]:
            # Eventhougn equals or empty list, n & Sp won't be affected
            if dominates(fit_i, fit_j):
                dominating_fits[fit_j] += 1  
                dominated_fits[fit_i].append(fit_j)  
            elif dominates(fit_j, fit_i):  
                dominating_fits[fit_i] += 1
                dominated_fits[fit_j].append(fit_i)
        if dominating_fits[fit_i] == 0: 
            current_front.append(fit_i)

    fronts = [[]]  # The first front
    for fit in current_front:
        fronts[-1].extend(map_fit_ind[fit])
    pareto_sorted = len(fronts[-1])

    # Rank the next front until all individuals are sorted or
    # the given number of individual are sorted.
    # If Sn=0 then the set of objectives belongs to the next front
    if not first_front_only:  # first front only
        N = min(len(fitness), k)
        while pareto_sorted < N:
            fronts.append([])
            for fit_p in current_front:
                # Iterate Sn in current fronts
                for fit_d in dominated_fits[fit_p]: 
                    dominating_fits[fit_d] -= 1  # Next front -> Sn - 1
                    if dominating_fits[fit_d] == 0:  # Sn=0 -> next front
                        next_front.append(fit_d)
                         # Count and append chromosomes with same objectives
                        pareto_sorted += len(map_fit_ind[fit_d]) 
                        fronts[-1].extend(map_fit_ind[fit_d])
            current_front = next_front
            next_front = []

    return np.array(fronts)

In [29]:
def CrowdingDist(fitness=None):
    """
    :param fitness: A list of fitness values
    :return: A list of crowding distances of chrmosomes
    
    The crowding-distance computation requires sorting the population according to each objective function value 
    in ascending order of magnitude. Thereafter, for each objective function, the boundary solutions (solutions with smallest and largest function values) 
    are assigned an infinite distance value. All other intermediate solutions are assigned a distance value equal to 
    the absolute normalized difference in the function values of two adjacent solutions.
    """

    # initialize list: [0.0, 0.0, 0.0, ...]
    distances = [0.0] * len(fitness)
    crowd = [(f_value, i) for i, f_value in enumerate(fitness)]  # create keys for fitness values

    n_obj = len(fitness[0])

    for i in range(n_obj):  # calculate for each objective
        crowd.sort(key=lambda element: element[0][i])
        # After sorting,  boundary solutions are assigned Inf 
        # crowd: [([obj_1, obj_2, ...], i_0), ([obj_1, obj_2, ...], i_1), ...]
        distances[crowd[0][1]] = float("Inf")
        distances[crowd[-1][1]] = float("inf")
        if crowd[-1][0][i] == crowd[0][0][i]:  # If objective values are same, skip this loop
            continue
        # normalization (max - min) as Denominator
        norm = float(crowd[-1][0][i] - crowd[0][0][i])
        # crowd: [([obj_1, obj_2, ...], i_0), ([obj_1, obj_2, ...], i_1), ...]
        # calculate each individual's Crowding Distance of i th objective
        # technique: shift the list and zip
        for prev, cur, next in zip(crowd[:-2], crowd[1:-1], crowd[2:]):
            distances[cur[1]] += (next[0][i] - prev[0][i]) / norm  # sum up the distance of ith individual along each of the objectives

    return distances

In [30]:
def closest_node(node, nodes):
    nodes = np.asarray(nodes)
    dist_2 = np.sum((nodes - node)**2, axis=1)
    return np.argmin(dist_2)

In [31]:
class args:
  pass

args.total_iters = 10 #100
args.eval_interval = 1 #10
args.init_w_type = 'none'
args.init_b_type = 'none'
args.learning_rate = 1e-1
args.weight_decay = 4e-5
args.momentum = 0.9 
args.eval_weights = [0.25,0.5,1.]
args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
args.train_weights = [0.25,0.5,1.]
args.config = "CONF_NB101"
args.workers = 2
args.pad = 'store_true'
args.input_size = 32 #cifar 32?
args.output_size = 8 #cifar 8?
args.last_channels = 64
args.batch_size = 16 #16
#CIFAR10
if Dataset_name == "Cifar-10":
  args.data_loc = '/content/drive/MyDrive/Github/NAS-PULITO/cifar-10-batches-py'
  args.dataset = 'cifar10'
  args.num_labels = 10

#CIFAR100
if Dataset_name == "Cifar-100":
  args.dataset = 'cifar100'
  args.data_loc = "/content/drive/MyDrive/NAS-PULITO/cifar-100-python"
  args.num_labels = 100

#IMAGENET
if Dataset_name == "ImageNet":
  args.dataset = 'ImageNet16-120'
  args.data_loc = "/content/drive/MyDrive/NAS/.torch/ImageNet16"
  args.num_labels = 120

args.init_channels = 16
args.search_space = 'nasbench201'

In [32]:
import pandas as pd

#CIFAR 10
if Dataset_name == "Cifar-10":
  df_iso = pd.read_csv('/content/drive/MyDrive/GitHub/NAS_project/naswot_cifar10_batch128_seed1_isomorf.csv')
  df_non_iso = pd.read_csv('/content/drive/MyDrive/GitHub/NAS_project/naswot_cifar10_batch128_completo.csv')
  df = df_non_iso.append(df_iso)

#CIFAR100
if Dataset_name == "Cifar-100":
  df_iso = pd.read_csv('/content/drive/MyDrive/GitHub/NAS_project/naswot_cifar100_batch128_seed1_isomorf.csv')
  df_non_iso = pd.read_csv('/content/drive/MyDrive/GitHub/NAS_project/naswot_cifar100_batch128_completo.csv')
  df = df_non_iso.append(df_iso)

#IMAGENET
if Dataset_name == "ImageNet":
  df = pd.read_csv('/content/drive/MyDrive/GitHub/NAS_project/naswot_ImageNet16-120_batch128_seed1_15625Arch.csv')


display(df)

,Unnamed: 0,arch_id,uniqArch,score,time,acc
0,0,0,(0)@avg_pool_3x3+(0)@avg_pool_3x3+(0)@nor_conv...,1281.548950,0.055238,28.211111
1,1,1,((0)@nor_conv_3x3)@avg_pool_3x3+(0)@nor_conv_3...,1380.631348,0.058419,44.488889
2,2,2,(((0)@avg_pool_3x3)@nor_conv_3x3+(0)@nor_conv_...,1263.264526,0.042976,27.633333
3,3,3,#+#+#+0,1047.412598,0.028810,29.433333
4,4,4,((0)@nor_conv_1x1+0)@nor_conv_1x1+0+0,1309.597046,0.041116,32.144444
...,...,...,...,...,...,...
15620,9155,15620,#+#+0,1054.735596,0.030721,29.822222
15621,9156,15621,#+((0)@avg_pool_3x3)@nor_conv_1x1+0,1227.618652,0.034945,31.800000
15622,9157,15622,#+((0)@nor_conv_3x3+(0)@nor_conv_3x3)@nor_conv...,1390.784668,0.049597,41.855555
15623,9158,15623,#+(#+(0)@avg_pool_3x3)@avg_pool_3x3+(0)@avg_po...,962.202087,0.030363,14.411111


In [33]:
display(df.sort_values(by=["acc"]))

,Unnamed: 0,arch_id,uniqArch,score,time,acc
13286,7642,13286,#+#+(#+#)@nor_conv_3x3,-inf,0.036226,0.833333
12746,7297,12746,#+#+#,1056.087158,0.050810,0.833333
13078,7506,13078,#+#+#,1056.087158,0.050810,0.833333
8354,4448,8354,#+#+#,1056.087158,0.050810,0.833333
632,156,632,#+#+(#+#)@avg_pool_3x3,-inf,0.038140,0.833333
...,...,...,...,...,...,...
8449,3936,8449,(#+((0)@nor_conv_3x3)@nor_conv_3x3)@nor_conv_1...,1423.089355,0.063976,46.500000
11777,5099,11777,#+(((0)@nor_conv_3x3)@nor_conv_3x3+(0)@nor_con...,1453.754639,0.063623,46.516667
3731,2003,3731,(((0)@nor_conv_3x3)@nor_conv_3x3+(0)@nor_conv_...,1447.406860,0.061208,46.555555
3888,2082,3888,(((0)@nor_conv_1x1)@nor_conv_3x3+(0)@nor_conv_...,1450.777466,0.066033,46.683333


In [34]:
args.config = eval(args.config)

In [35]:
evaluator = Evaluator(args)
task = CVTask(args)
model_builder = ModelBuilder(args)

In [36]:
df_config = pd.read_csv("conf.csv")

In [37]:
searchspace = nasspace.get_search_space(dataset, df_config.conf)
train_loader = datasets.get_data(dataset, data_loc, batch_size)

data_iterator = iter(train_loader)
input, target = next(data_iterator)
input, target = input.to(device), target.to(device)

[2022-08-28 19:17:02] Try to use the default NATS-Bench (topology) path from fast_mode=True and path=None.


# Caloclo ET-NAS

In [ ]:
from numpy.random import default_rng

Number_experiments = 30
filename = f'ET-NAS_on_{dataset}_in_{Number_experiments}_experimets.csv'
arch_dict = []

arch_info = ['arch_id' , 'time', 'acc', 'test_acc']
history = []
from numpy.random import default_rng
population_size=20 
n_sample=5
cycles= 60

for j in range(0,Number_experiments):
  print(j)




  new_gen = []
  history = []


  start = timer()

  uid_population =  random.sample(range(len(searchspace)),population_size)

  population =  get_uid_and_measures(uid_population, input, searchspace, "log_synflow") 

  history = get_archs(searchspace, uid_population)

          
  while len(history) <= cycles:

      uid_sample = random.sample(range(len(population)),n_sample)

      sample = population[uid_sample]

      parent = max(sample[sample[:,2] == max(sample[:,2])], key=lambda i: i[1]) 

      child_idx = searchspace.mutate_arch(int(parent[0])) 

      unique_str_child = searchspace.get_unique_str(child_idx)

      degree_child = NNDegree(searchspace, child_idx)

      degree_parent = parent[-1]

      if unique_str_child not in history and degree_child >= degree_parent:


        uid_and_measure = np.array(get_uid_and_measure(child_idx, input, "log_synflow", population))

        new_gen.append([int(uid_and_measure[0]), uid_and_measure[1], uid_and_measure[2] ])

        population = np.delete(population, 0, axis=0)

        population = np.vstack((population, uid_and_measure))

        history.append(unique_str_child)


     
          
          
  

  gen = np.array(new_gen)

  gen = gen[gen[:,2] == max(gen[:,2])] #consider the architectures with higher NNdegree

  measure= sum_syn_naswot_degree(gen, input, searchspace,task,model_builder,evaluator) 

  syn_naswot = measure[:,[2,3]]

  pareto_index = sortNondominated(syn_naswot, first_front_only=True)
  syn_naswot_pareto = syn_naswot[pareto_index[0]]
  arch_pareto = measure[pareto_index[0]][:,0]


  regression = []


  if len(arch_pareto) > 1:
    for uid in arch_pareto:
      regression.append(get_regression_score(uid, task, model_builder,evaluator))
    
    syn_naswot_regression = np.hstack((syn_naswot_pareto, np.array(regression).reshape(-1,1)))
    measures_normalized = (syn_naswot_regression) / (syn_naswot_regression.max(axis = 0))
    best_arch_index = closest_node(np.array([1,1,1]), measures_normalized)
    best_arch = arch_pareto[best_arch_index]
    total_time_cost = (timer()-start)

  else:
    best_arch = arch_pareto[0]
    total_time_cost = (timer()-start)


  arch_dict.append({'arch_id' : int(best_arch), 'time': total_time_cost, 'acc' : df.set_index('arch_id').loc[int(best_arch), 'acc'], 'test_acc': searchspace.get_final_accuracy(int(best_arch)) })

  with open(filename, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = arch_info)
    writer.writeheader()
    writer.writerows(arch_dict)






In [39]:
OurRea_on_ImageNet16_120 = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/ET-NAS_on_ImageNet16-120_in_30_experimets.csv")
print(OurRea_on_ImageNet16_120.mean())
print(OurRea_on_ImageNet16_120.std())

arch_id     8368.133333
time          26.992938
acc           46.052963
test_acc      46.317778
dtype: float64
arch_id     3934.282380
time           4.191018
acc            0.458222
test_acc       0.214553
dtype: float64
